**Notebook preparation and importing libraries**

In [1]:
from pathlib import Path
from hvac import Quantity
from hvac.fluids import Fluid, HumidAir, CoolPropWarning
from hvac.vapor_compression import VariableSpeedCompressor
from hvac.vapor_compression.machine_bis import SingleStageVaporCompressionMachine
from hvac.heat_transfer.heat_exchanger.fin_tube.air_evaporator import PlainFinTubeCounterFlowEvaporator
from hvac.heat_transfer.heat_exchanger.fin_tube.air_condenser import PlainFinTubeCounterFlowCondenser

In [2]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=CoolPropWarning)

In [3]:
from hvac.logging import ModuleLogger
logger = ModuleLogger.get_logger(__name__)
logger.setLevel(ModuleLogger.INFO)

In [4]:
Q_ = Quantity

# Rating of a Single-Stage Vapor Compression Machine

A single-stage vapor compression machine is composed of a compressor, an evaporator, a condenser, and an expansion device. Air at a given state and mass flow rate enters the evaporator. At the condenser, air also enters at a given state and mass flow rate. The expansion device regulates the mass flow rate of refrigerant in order to maintain the set degree of superheat. The compressor (either with fixed or variable speed drive) runs at a given compressor speed. The question to answer here is what the operating point of the vapor compression machine will be under the given operating conditions.

## Configuration of the Vapor Compression Machine

To configure a single-stage vapor compression machine, we first need to define compressor, evaporator and condenser.

**Compressor**<br>
Based on preliminary calculations (see the first notebook of this series), a variable speed compressor with refrigerant R134a has been selected using selection software of the compressor manufacturer. The polynomial coefficients of the functions describing the performance quantities of this compressor were saved in a csv-file.   

First, define the refrigerant:

In [5]:
R134a = Fluid('R134a')

Now we can define the compressor:

In [6]:
compressor = VariableSpeedCompressor(
    coeff_file=Path("compressor_data/VTZ038-G_R134a.csv"),
    refrigerant_type=R134a,
    units={'m_dot': 'kg / hr', 'speed': '1 / s'}
)

**Evaporator**<br>
We will use the same air evaporator defined in notebook no. 2 (plain fin-tube counterflow evaporator).

In [7]:
evaporator = PlainFinTubeCounterFlowEvaporator(
    L1=Q_(0.731, 'm'),
    L3=Q_(0.244, 'm'),
    N_r=2,
    S_t=Q_(25.4, 'mm'),
    S_l=Q_(22.0, 'mm'),
    D_i=Q_(8.422, 'mm'),
    D_o=Q_(10.2, 'mm'),
    t_f=Q_(0.3302, 'mm'),
    N_f=1 / Q_(3.175, 'mm')
)

**Condenser**<br>
We will use the same air condenser defined in notebook no. 3 (plain fin-tube counterflow condenser).

In [8]:
condenser = PlainFinTubeCounterFlowCondenser(
    L1=Q_(0.909, 'm'),
    L3=Q_(0.303, 'm'),
    N_r=4,
    S_t=Q_(25.4, 'mm'),         
    S_l=Q_(22.0, 'mm'),         
    D_i=Q_(8.422, 'mm'),        
    D_o=Q_(10.2, 'mm'),         
    t_f=Q_(0.3302, 'mm'),       
    N_f=1 / Q_(3.175, 'mm')    
)

**Vapor compression machine**<br>
Configuration of the vapor compression machine is simple. You just need to pass the evaporator, condenser, compressor, and refrigerant to the `SingleStageVaporCompressionMachine` constructor:

In [9]:
machine = SingleStageVaporCompressionMachine(
    evaporator, condenser, compressor, R134a,
    n_cmp_min=Q_(2100, '1 / min'),
    n_cmp_max=Q_(5400, '1 / min'),
    logger=logger
)

## Specify Known Operating Conditions

The following inputs will unambiguously determine the operation of the vapor compression machine:
- mass flow rate of air through the evaporator
- mass flow rate of air through the condenser
- air state at evaporator inlet
- air state at condenser inlet
- degree of superheat set on the expansion device
- compressor speed (only in case of a variable speed compressor)

In [10]:
machine.set_operating_conditions(
    evp_m_dot_air=Q_(1500, 'kg / hr'),
    cnd_m_dot_air=Q_(2233.861, 'kg / hr'),
    evp_air_in=HumidAir(Tdb=Q_(24.0, 'degC'), RH=Q_(50, 'pct')),
    cnd_air_in=HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct')),
    dT_sh=Q_(10, 'K'),
    n_cmp=Q_(4000, '1 / min')
)

## Steady-State Operation

To determine the steady-state operation of the machine a similar, iterative solving technique is used as explained in Chapter 14 of the book *Stoecker, W. F., & Jones, J. W. (1982). Refrigeration and air conditioning. McGraw-Hill International Editions, Mechanical Technology Series.* Three rating methods are implemented in the single-stage vapor compression machine model:
- Method `rate` uses a for-loop that will break when the the deviation between the refrigerant mass flow rate according to the compressor model and the refrigerant mass flow rate according to the evaporator is smaller than `abs_tol` (absolute tolerance) or `rel_tol` (relative tolerance).  
- Method `rate_by_root_finding` uses a root-finding algorithm from scipy (`scipy.optimize.root`). Normally, method `rate_by_root_finding` will work better. However, it can fail by guessing values for the evaporation and condensing temperature which are physically impossible. Also, one should always check the mass and energy balance afterwards to see if the result can truly be valid.
- Method `rate_by_minimization` uses a minimization algorithm from scipy (`scipy.optimize.minimize`). It tries to find the evaporation and condensation temperature for which the deviation between the refrigerant mass flow rate according to the compressor model and the refrigerant mass flow rate according to the evaporator is minimal. As physical bounds can be set with this algorithm, the risk that the algorithm will fail is smaller. However, this algorithm seems to be slower than the root-finding algorithm used in `rate_by_root_finding`.

In [11]:
%%time 
# machine.rate(
#   T_evp_ini=Q_(5.0, 'degC'), 
#   T_cnd_ini=Q_(50.0, 'degC'), 
#   abs_tol=Q_(1, 'kg / hr')
# )

# machine.rate_by_root_finding(
#   T_evp_ini=Q_(5.0, 'degC'), 
#   T_cnd_ini=Q_(50.0, 'degC')
# )

machine.rate_by_minimization(
    T_evp_ini=Q_(5.0, 'degC'), 
    T_cnd_ini=Q_(50.0, 'degC')
)

[13288 | __main__ | INFO] Iteration 1: try with T_evp = 5.000 °C and T_cnd = 50.000 °C
[13288 | __main__ | INFO] Iteration 1: deviation with 5.000 °C and 50.000 °C: 2.636 kg/h
[13288 | __main__ | INFO] Iteration 2: try with T_evp = 5.250 °C and T_cnd = 50.000 °C
[13288 | __main__ | INFO] Iteration 2: deviation with 5.250 °C and 50.000 °C: 5.651 kg/h
[13288 | __main__ | INFO] Iteration 3: try with T_evp = 5.000 °C and T_cnd = 52.500 °C
[13288 | __main__ | INFO] Iteration 3: deviation with 5.000 °C and 52.500 °C: 2.372 kg/h
[13288 | __main__ | INFO] Iteration 4: try with T_evp = 4.750 °C and T_cnd = 52.500 °C
[13288 | __main__ | INFO] Iteration 4: deviation with 4.750 °C and 52.500 °C: 0.932 kg/h
[13288 | __main__ | INFO] Iteration 5: try with T_evp = 4.500 °C and T_cnd = 53.750 °C
[13288 | __main__ | INFO] Iteration 5: deviation with 4.500 °C and 53.750 °C: 4.495 kg/h
[13288 | __main__ | INFO] Iteration 6: try with T_evp = 4.750 °C and T_cnd = 55.000 °C
[13288 | __main__ | INFO] Iterati

CPU times: total: 5min 9s
Wall time: 5min 32s


**Check mass balance**<br>
Check the difference of the mass flow rate of refrigerant according to the evaporator model with respect to the mass flow rate of refrigerant according to the compressor model.

In [12]:
abs_err_mb, rel_err_mb = machine.check_mass_balance()
print(
    "mass balance: "
    f"absolute error = {abs_err_mb.to('kg / hr'):~P.3f}, "
    f"relative error = {rel_err_mb.to('pct'):~P.3f}"
)

mass balance: absolute error = 0.053 kg/h, relative error = 0.054 %


**Check energy balance**<br>
Check the difference of the refrigeration capacity (heat absorption rate) according to the evaporator model with respect to the refrigeration capacity according to the compressor model (which is calculated as the difference between the heat rejection rate of the condenser and the compressor power).

In [13]:
abs_err_eb, rel_err_eb = machine.check_energy_balance()
print(
    "energy balance: "
    f"absolute error = {abs_err_eb.to('kW'):~P.3f}, "
    f"relative error = {rel_err_eb.to('pct'):~P.2f}"
)

energy balance: absolute error = 0.004 kW, relative error = 0.10 %


**Results**

The operating point of the vapor compression machine is defined by the evaporating and condensing temperature:

In [14]:
print(
    f"evaporation temperature = {machine.Te.to('degC'):~P.3f}\n"
    f"condensing temperature = {machine.Tc.to('degC'):~P.3f}"
)

evaporation temperature = 4.957 °C
condensing temperature = 57.256 °C


Other operating characteristics are:

In [15]:
print(
    f"heat absorption rate = {machine.Qc_dot.to('kW'):~P.3f}\n"
    f"heat rejection rate = {machine.Qh_dot.to('kW'):~P.3f}\n"
    f"compressor power = {machine.Wc_dot.to('kW'):~P.3f}\n"
    f"COP = {machine.COP.to('frac'):~P.2f}\n"
    f"refrigerant mass flow rate = {machine.m_dot.to('kg / hr'):~P.3f}"
)

heat absorption rate = 4.101 kW
heat rejection rate = 5.882 kW
compressor power = 1.777 kW
COP = 2.06 frac
refrigerant mass flow rate = 97.972 kg/h


The state of air at the outlet of evaporator and condenser:

In [16]:
print(
    f"evaporator air out = {evaporator.air_out.Tdb.to('degC'):~P.2f} DB, "
    f"{evaporator.air_out.RH.to('pct'):~P.1f} RH\n"
    f"condenser air out = {condenser.air_out.Tdb.to('degC'):~P.2f} DB, "
    f"{condenser.air_out.RH.to('pct'):~P.1f} RH"
)

evaporator air out = 17.43 °C DB, 65.2 % RH
condenser air out = 44.29 °C DB, 18.3 % RH


The different states of refrigerant along the vapor compression cycle:

In [17]:
print(
    f"suction gas temperature = {machine.suction_gas.T.to('degC'):~P.2f}\n"
    f"discharge gas temperature = {machine.discharge_gas.T.to('degC'):~P.2f}\n"
    f"liquid temperature = {machine.liquid.T.to('degC'):~P.2f}\n"
    f"liquid/vapor mixture temperature = {machine.mixture.T.to('degC'):~P.2f}, "
    f"vapor quality = {machine.mixture.x.to('frac'):~P.2f}\n"
    f"subcooling degree = {machine.sub_cooling.to('K'):~P.2f}"
)

suction gas temperature = 14.96 °C
discharge gas temperature = 99.76 °C
liquid temperature = 42.27 °C
liquid/vapor mixture temperature = 4.96 °C, vapor quality = 0.27 frac
subcooling degree = 14.99 K


Air-side pressure drop across evaporator and condenser:

In [18]:
print(
    f"evaporator air-side pressure drop = {evaporator.dP_air.to('Pa'):~P.1f}\n"
    f"condenser air-side pressure drop = {condenser.dP_air.to('Pa'):~P.1f}"
)

evaporator air-side pressure drop = 14.4 Pa
condenser air-side pressure drop = 32.1 Pa
